In [1]:
print("HI")

HI


In [2]:
import os


def find_move_files(directory):
    move_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".move"):
                move_files.append(os.path.join(root, file))
    return move_files


# Use Sui's Examples
directory = "./sui/sui_programmability/examples/"
move_files = find_move_files(directory)

In [3]:
import ssl
from elasticsearch import Elasticsearch

from dotenv import dotenv_values
config = dotenv_values(".env")

# Elasticsearch Config
es = Elasticsearch(
    ['https://localhost:9200'],
    http_auth=('elastic', config['ELASTIC_PASSWORD']),
    verify_certs=True,
    ca_certs='http_ca.crt'
)

# Check Connection
if not es.ping():
    raise ValueError("Fail to connect to Elasticsearch")
else:
    print("Connect to Elasticsearch Success")


Connect to Elasticsearch Success


/tmp/ipykernel_636675/2498718443.py:8: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [8]:
index_name = "sui_gpt"
# Define all queries
try:
    query = {
        "query": {
            "match_all": {}
        }
    }

    # Clear index: Delete By Query
    es.delete_by_query(index=index_name, body=query)
except:
    pass

In [9]:

# 建立索引
es.indices.create(index=index_name, ignore=400)


/tmp/ipykernel_636675/2785200631.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, ignore=400)


ObjectApiResponse({'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [sui_gpt/Dp9JFZDxTmqXoYPXmKdQ9A] already exists', 'index_uuid': 'Dp9JFZDxTmqXoYPXmKdQ9A', 'index': 'sui_gpt'}], 'type': 'resource_already_exists_exception', 'reason': 'index [sui_gpt/Dp9JFZDxTmqXoYPXmKdQ9A] already exists', 'index_uuid': 'Dp9JFZDxTmqXoYPXmKdQ9A', 'index': 'sui_gpt'}, 'status': 400})

In [16]:
for move_file in tqdm(move_files):
    (
        move_file
    ) = "./sui/sui_programmability/examples/fungible_tokens/sources/managed.move"
    with open(move_file, "r") as f:
        d = (f.readlines())

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 33156.55it/s]


In [21]:
import json
print(json.dumps(["".join(d)]))

["// Copyright (c) Mysten Labs, Inc.\n// SPDX-License-Identifier: Apache-2.0\n\n/// Example coin with a trusted manager responsible for minting/burning (e.g., a stablecoin)\n/// By convention, modules defining custom coin types use upper case names, in contrast to\n/// ordinary modules, which use camel case.\nmodule fungible_tokens::managed {\n    use std::option;\n    use sui::coin::{Self, Coin, TreasuryCap};\n    use sui::transfer;\n    use sui::tx_context::{Self, TxContext};\n\n    /// Name of the coin. By convention, this type has the same name as its parent module\n    /// and has no fields. The full type of the coin defined by this module will be `COIN<MANAGED>`.\n    struct MANAGED has drop {}\n\n    #[allow(unused_function)]\n    /// Register the managed currency to acquire its `TreasuryCap`. Because\n    /// this is a module initializer, it ensures the currency only gets\n    /// registered once.\n    fun init(witness: MANAGED, ctx: &mut TxContext) {\n        // Get a treasury

In [10]:
from tqdm import tqdm

for move_file in tqdm(move_files):
    # Read files
    with open(move_file + "a", "r") as f:
        source_move = f.read()
    with open(move_file + "a", "r") as f:
        annotated_move = f.read()
    with open(move_file + "ac", "r") as f:
        annotated_commented_move = f.read()
    with open(move_file + ".summary", "r") as f:
        code_summary = f.read()
    with open(move_file + ".title", "r") as f:
        code_title = f.read()

    # create doc
    doc = {
        "source_move": source_move,
        "annotated_move": annotated_move,
        "annotated_commented_move": annotated_commented_move,
        "code_summary": code_summary,
        "code_title": code_title
    }

    # 索引文檔
    es.index(index=index_name, document=doc)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 121.52it/s]


In [11]:
instructions = "A fungible coin name and symbol is Eason that I can manage to airdrop by a list of whitelists"

query = {
    "query": {
        "more_like_this": {
            "fields": ["code_summary", "code_title"],
            "like": instructions,
            "min_term_freq": 1,
            "max_query_terms": 12
        }
    }
}

# 執行 MLT 查詢
results = es.search(index=index_name, body=query)


# Assuming 'results' is the result obtained from querying Elasticsearch

# Create a dictionary to store the extracted fields
result_dict = {}

# Iterate through each document in the Elasticsearch results
for doc in results['hits']['hits']:
    # Extract the 'code_title' field
    title = doc['_source']['code_title']

    # Extract other fields
    source_move = doc['_source']['source_move']
    annotated_move = doc['_source']['annotated_move']
    annotated_commented_move = doc['_source']['annotated_commented_move']
    code_summary = doc['_source']['code_summary']
    
    # Store the extracted field values in a dictionary
    result_dict[title] = {
        'source_move': source_move,
        'annotated_move': annotated_move,
        'annotated_commented_move': annotated_commented_move,
        'code_summary': code_summary,
        'code_title': title
    }

# Print the resulting dictionary

# Take the top 3 items
top_3_results = dict(list(result_dict.items())[:3])




output = {
    "match_titles": result_dict.keys(),
    "top3_match": top_3_results.keys(),
    "top3_match": top_3_results,
}

In [12]:
# Find managed fungible coin and airdrop NFT contract first! Fit our need!
print(result_dict.keys())

dict_keys(['Managed Fungible Tokens', 'Cross-Chain Airdrops of NFTs', 'Marketplace Implementation', 'NFT Module', 'Capy Admin', 'Capy Marketplace', 'Treasury Lock Pattern', 'Capy Item Store', 'Module for a regulated coin type with balance accessors for implementing a RegulatedCoin interface', 'Auction Tests'])
